In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Product_Id'].tolist()

总数量：150


['988398',
 '4316658',
 '1093509',
 '4098330',
 '4570446',
 '1144742',
 '1127436',
 '4234518',
 '4234508',
 '3648954',
 '1075829',
 '4316295',
 '4234520',
 '3642704',
 '4091955',
 '4118028',
 '4158821',
 '56878',
 '1051673',
 '3648983',
 '1051674',
 '987848',
 '4406144',
 '4304217',
 '4091935',
 '4091728',
 '1127672',
 '4310099',
 '4203438',
 '937462',
 '4407489',
 '3644678',
 '56657',
 '4578759',
 '4401497',
 '4309124',
 '4307410',
 '4118006',
 '4094530',
 '45119',
 '4091719',
 '988750',
 '4329104',
 '989081',
 '989080',
 '46157',
 '44595',
 '4570440',
 '4570327',
 '4118022',
 '3642758',
 '3652444',
 '4091934',
 '4312336',
 '1020670',
 '988801',
 '1051804',
 '4407357',
 '4091723',
 '1000392',
 '55421',
 '43454',
 '3644426',
 '1070086',
 '4307176',
 '4303644',
 '4092407',
 '4311398',
 '1070229',
 '46155',
 '4118044',
 '1040042',
 '46156',
 '4203450',
 '56656',
 '4309328',
 '4405831',
 '4305427',
 '4303349',
 '988694',
 '55369',
 '4304190',
 '986633',
 '56659',
 '3644700',
 '4407355',
 

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get('https://www.atp-autoteile.de/de/product/vehicles/' + input_.loc[a, 'Product_Id'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<section class="matching-vehicles   row">\r\n    <div class="col-md-6 col-xs-12">\r\n            <div class="accordion-vehicles " data-tab="9953" data-manufacturer-selector> BMW</div>\r\n            <div class="tab-content hidden" id="9953" data-product-matching-vehicles-9953>\r\n               \r\n            </div>\r\n            <hr/>\r\n            </div>\r\n\r\n</section>'

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./make.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <section class="matching-vehicles row">
   <div class="col-md-6 col-xs-12">
    <div class="accordion-vehicles" data-manufacturer-selector="" data-tab="9953">
     BMW
    </div>
    <div class="tab-content hidden" data-product-matching-vehicles-9953="" id="9953">
    </div>
    <hr/>
   </div>
  </section>
 </body>
</html>



In [8]:
list_make = [make.strip() for make in html.xpath('//div[@data-manufacturer-selector]/text()')]
        
list_make

['BMW']

In [9]:
list_make_code = [code.strip() for code in html.xpath('//div[@data-manufacturer-selector]/@data-tab')]
        
list_make_code

['9953']

In [10]:
df_temp = pd.DataFrame({'No.': int(input_.loc[a, '序号']),
                        'Product_Id': input_.loc[a, 'Product_Id'],
                        'Make': list_make,
                        'Make_Code': list_make_code})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'序号': int(input_.loc[a, '序号']),
                             'Product_Id': input_.loc[a, 'Product_Id']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [13]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Make_Code'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./test_make.xlsx', index=False)

output_correct

,No.,Product_Id,Make,Make_Code
0,1,988398,BMW,9953


In [14]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_make_error.xlsx', index=False)

output_error

""
